# Import Requirements

In [1]:
from __future__ import division
import sys
import pysam
import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import time
from collections import defaultdict
import csv
from itertools import islice
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sys.path.append('/Users/greg/damlarces/')
from damlarces.NGSTools import ReadTools

# How to read in filepaths

In [3]:
files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')
f = files[0]
print len(files)

96


# Read in BAM files and count reads

In [10]:
start=5800
stop=6060

def count_reads_for_gene(start, stop):
    
    hiv_bams = {'filepath':[], 'numberReads':[]}

    for filepath in glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam'):
        samfile = pysam.AlignmentFile(filepath, 'rb')
        numreads = 0
        for read in samfile.fetch('HXB2', start, stop):
            if ~read.is_qcfail:
                numreads+=1
        if 'Frag' not in filepath:
            if numreads >= 1000:
                #print filepath, numreads
                hiv_bams['filepath'].append(filepath)
                hiv_bams['numberReads'].append(numreads)

    hivBAM_df = pd.DataFrame(hiv_bams)
    return hivBAM_df

tat1_df = count_reads_for_gene(5831, 6045)
print tat1_df.shape

(52, 2)


In [11]:
tat1_df.head()

,filepath,numberReads
0,/Users/greg/Desktop/FullNeuroIllumina/BAMs/A00...,114761
1,/Users/greg/Desktop/FullNeuroIllumina/BAMs/A00...,2382
2,/Users/greg/Desktop/FullNeuroIllumina/BAMs/A00...,3913
3,/Users/greg/Desktop/FullNeuroIllumina/BAMs/A00...,20365
4,/Users/greg/Desktop/FullNeuroIllumina/BAMs/A00...,2023


# Process all Tat 1 files

In [15]:
#files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')
for f in tat1_df['filepath']:
    patient = f.split('/')[-1].split('.')[0].split('-')[0]
    visit = f.split('/')[-1].split('.')[0].split('-')[1]

In [16]:
files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')

meta_data = {'Patient':[], 'Visit':[], 'total_reads':[], 'right_length':[], 'no_stop':[]}

for f in tat1_df['filepath']:
    patient = f.split('/')[-1].split('.')[0].split('-')[0]
    visit = f.split('/')[-1].split('.')[0].split('-')[1]
    print patient, visit

    samfile = pysam.Samfile(f, mode='rb').fetch(region='HXB2:5800-6060')

    # time
    t = time.time()

    counts = defaultdict(int)
    it = ReadTools.SeqReadRecord.translate_from_stream(samfile, [('Tat1', 5829, 6046)])
    total_reads=0
    right_length=0
    no_stop=0

    all_len = []
    for orf in it: 
        # count total reads
        total_reads += 1 
        # verify the correct ORF length
        if len(orf.seq)==72:
            right_length += 1
            # check that there is not a premature stop codon
            if '*' not in orf.seq:
                no_stop+=1
                # iterate through the amino acid sequences
                for i, aa in enumerate(orf.seq):
                    pos = i+1
                    # ignore the padded gaps
                    if aa != '-':
                        counts[(patient, visit, orf.chrom, pos, aa)] += 1

    # time
    elapsed = time.time() - t
    print 'Time:', elapsed

    meta_data['Patient'].append(patient)
    meta_data['Visit'].append(visit)
    meta_data['total_reads'].append(total_reads)
    meta_data['right_length'].append(right_length)
    meta_data['no_stop'].append(no_stop)
    
    out_file = f.replace('hivsorted.bam', 'AAcounts.csv')
    out_file = out_file.replace('/FullNeuroIllumina/BAMs', '/FullNeuroIllumina/AAcounts')
    print out_file
    
    df = pd.Series(counts)
    df.index.names = ['Patient', 'Visit', 'ORF', 'Position', 'AA']
    df = df.reset_index()
    df.rename(columns={0:'Count'}, inplace=True)
    df.to_csv(out_file, index=False)

A0001 R09
Time: 162.457543135
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0001-R09.AAcounts.csv
A0013 R09
Time: 3.56672000885
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0013-R09.AAcounts.csv
A0019 R12
Time: 5.7593460083
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0019-R12.AAcounts.csv
A0034 R04
Time: 29.7700800896
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0034-R04.AAcounts.csv
A0044 R10
Time: 2.98477005959
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0044-R10.AAcounts.csv
A0045 R03
Time: 6.15661096573
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0045-R03.AAcounts.csv
A0059 R08
Time: 105.654245853
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0059-R08.AAcounts.csv
A0082 R06
Time: 53.0769300461
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0082-R06.AAcounts.csv
A0083 R06
Time: 4.79311299324
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0083-R06.AAcounts.csv
A0109 R11
Time: 126.890389919
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0109-R11.AAcounts.csv
A

In [17]:
pd.DataFrame(meta_data)

,Patient,Visit,no_stop,right_length,total_reads
0,A0001,R09,37923,110987,111132
1,A0013,R09,994,2336,2339
2,A0019,R12,926,3867,3872
3,A0034,R04,7197,16298,19779
4,A0044,R10,841,1931,1931
5,A0045,R03,1626,4014,4029
6,A0059,R08,33851,68998,69028
7,A0082,R06,13644,31880,34512
8,A0083,R06,1351,3057,3078
9,A0109,R11,43039,82254,82316


# Process with stop codons?

# Translation processing

In [51]:
aa_countQC = {'csvfile':[],
              'positions':[],
              'avg_coverage':[],
              'max_coverage':[],
              'min_coverage':[],
              'coverage > 100':[],
              'coverage > 1000': []}


csv_files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/AAcounts/*.csv')
for csv_file in csv_files:
    test_df = pd.read_csv(csv_file)
    
    aa_coverage = []
    for i, group in test_df.groupby(['Position']):
        #print i, sum(group['Count'])
        aa_coverage.append(sum(group['Count']))

    # aa_coverage metadata
    aa_countQC['csvfile'].append(csv_file)
    aa_countQC['positions'].append(len(aa_coverage))
    aa_countQC['avg_coverage'].append(np.mean(aa_coverage))
    aa_countQC['max_coverage'].append(max(aa_coverage))
    aa_countQC['min_coverage'].append(min(aa_coverage))
    aa_countQC['coverage > 100'].append(len([i for i in aa_coverage if i>100]))
    aa_countQC['coverage > 1000'].append(len([i for i in aa_coverage if i>1000]))

In [53]:
QCdf = pd.DataFrame(aa_countQC)
QCdf.to_csv('/Users/greg/Desktop/QC.csv', index=False)

In [87]:
QCdf

,avg_coverage,coverage > 100,coverage > 1000,csvfile,max_coverage,min_coverage,positions
0,5427.736111,72,51,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,18841,154,72
1,235.000000,48,0,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,755,2,72
2,156.708333,25,0,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,708,6,72
3,1197.169014,66,29,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,3563,1,71
4,180.361111,53,0,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,456,4,72
5,413.000000,60,0,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,843,19,72
6,8439.361111,72,68,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,17832,230,72
7,3530.527778,72,66,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,6536,113,72
8,309.944444,58,0,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,851,5,72
9,11304.638889,72,72,/Users/greg/Desktop/FullNeuroIllumina/AAcounts...,23780,2443,72


In [95]:
Counter(QCdf['positions'])

Counter({42: 1, 64: 1, 71: 2, 72: 48})

# Get consensus amino acid

In [93]:
csv_files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/AAcounts/*.csv')
print csv_files[1]
df = pd.read_csv(csv_files[3])
df.head()

for i, group in df.groupby(['Position'])[['AA','Count']]: 
    #print group
    idx = group['Count'].idxmax()
    #print idx
    max_df = df.loc[idx, ['Position', 'AA', 'Count']]
    print max_df['Position'], max_df['AA']

/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0013-R09.AAcounts.csv
1 M
2 E
3 P
4 V
5 D
6 P
7 R
8 L
9 E
10 P
11 W
12 K
13 H
14 P
15 G
16 S
17 Q
18 P
19 M
20 T
21 P
22 C
23 T
24 N
25 C
26 Y
27 C
28 K
29 K
30 V
31 L
32 L
33 S
34 L
35 P
36 S
37 L
38 F
39 H
40 N
41 K
42 R
43 L
44 R
45 H
46 L
47 L
48 W
49 Q
50 E
51 E
52 A
53 E
54 T
55 A
56 T
57 K
58 S
60 S
61 K
62 Q
63 S
64 D
65 S
66 S
67 S
68 F
69 S
70 I
71 K
72 A


In [59]:

#df = df.groupby(['Position','AA'])['Count'].agg({'Count': 'sum'})
#df = df.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

for pos in df.groupby('Position'):
    print pos[0], sum(pos[1]['Count'])

1 18841
2 18841
3 18267
4 17701
5 16875
6 15506
7 14700
8 13438
9 12349
10 10865
11 10327
12 9585
13 8947
14 8553
15 8087
16 7139
17 6860
18 6457
19 506
20 495
21 486
22 472
23 165
24 154
25 193
26 198
27 198
28 211
29 645
30 1112
31 1324
32 1683
33 1895
34 2157
35 2286
36 2325
37 2376
38 2373
39 2392
40 2404
41 2681
42 2695
43 2852
44 2903
45 2901
46 2918
47 1809
48 1799
49 1640
50 1586
51 1420
52 1358
53 1325
54 1276
55 911
56 880
57 836
58 271
59 246
60 227
61 225
62 233
63 234
64 673
65 10314
66 11408
67 11898
68 13249
69 13903
70 15227
71 15756
72 15755
